# NiN

In [1]:
from tensorflow import keras
import numpy as np
import tensorflow as tf

from d2l.tensorflow import config, data, plot

config.setup('CPU')

Tensorflow running on CPU


## NiN block

In [2]:
def nin_block(num_channels, kernel_size, strides, padding):
    blk = keras.models.Sequential()
    keras.layers.ZeroPadding2D(padding=padding)
    blk.add(keras.layers.Conv2D(
        filters=num_channels, kernel_size=kernel_size, strides=strides, activation='relu'
    ))
    blk.add(keras.layers.Conv2D(filters=num_channels, kernel_size=1, activation='relu'))
    blk.add(keras.layers.Conv2D(filters=num_channels, kernel_size=1, activation='relu'))

    return blk

## NiN model

In [3]:
nin = keras.models.Sequential()

nin.add(nin_block(num_channels=96, kernel_size=11, strides=4, padding=0))
nin.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same'))
nin.add(nin_block(num_channels=256, kernel_size=5, strides=1, padding=2))
nin.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same'))
nin.add(nin_block(num_channels=384, kernel_size=3, strides=1, padding=1))
nin.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same'))

nin.add(keras.layers.Dropout(rate=0.5))
nin.add(nin_block(num_channels=10, kernel_size=3, strides=1, padding=1))
nin.add(keras.layers.GlobalAvgPool2D())

nin.add(keras.layers.Flatten())

nin.build((None, 224, 224, 1))

In [4]:
nin.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_1 (Sequential)    multiple                  30336     
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
sequential_2 (Sequential)    multiple                  746240    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
sequential_3 (Sequential)    multiple                  1180800   
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 multiple                  0         
_________________________________________________________________
dropout (Dropout)            multiple                  0

In [5]:
X = np.random.uniform(size=(1, 224, 224, 1)).astype(np.float32)

for layer in nin.layers:
    X = layer(X)
    print('{} output shape: {}'.format(layer.name, X.shape))

sequential_1 output shape: (1, 54, 54, 96)
max_pooling2d output shape: (1, 27, 27, 96)
sequential_2 output shape: (1, 23, 23, 256)
max_pooling2d_1 output shape: (1, 12, 12, 256)
sequential_3 output shape: (1, 10, 10, 384)
max_pooling2d_2 output shape: (1, 5, 5, 384)
dropout output shape: (1, 5, 5, 384)
sequential_4 output shape: (1, 3, 3, 10)
global_average_pooling2d output shape: (1, 10)
flatten output shape: (1, 10)


## Training

In [6]:
batch_size = 128
epochs = 5
learning_rate = 0.1

optmizer = keras.optimizers.SGD(learning_rate=learning_rate)
loss = keras.losses.CategoricalCrossentropy()
metric = keras.metrics.CategoricalAccuracy()

nin.compile(optimizer=optmizer, loss=loss, metrics=metric)

In [7]:
X_train, y_train, X_test, y_test = data.load_fashion_mnist_keras()

train_iter = data.resize_images_generator(X_train, y_train, batch_size, [224, 224])
test_iter = data.resize_images_generator(X_test, y_test, batch_size, [224, 224])

train_steps = len(X_train) // batch_size + 1
val_steps = len(X_test) // batch_size + 1

In [8]:
# hist = nin.fit(train_iter, epochs=epochs, validation_data=test_iter, steps_per_epoch=train_steps, validation_steps=val_steps)

In [9]:
# plot.plot_history(hist, 'categorical_accuracy')